In [360]:
import mne
import os
import glob
import numpy as np
import pandas as pd
from scipy.io import savemat, loadmat
import matplotlib.pyplot as plt
import pickle

from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs
from mne_icalabel import label_components

import sys
sys.path.append('/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/utils')
from pp_utils import *
from plot_utils import *


In [361]:
# def mapchange_keystrokes_3(t_modeswitch, t_keystroke):
#     """ 
#     Finds all the keystroke triggers that are the first keystrokes after a map change.

#     t_modeswitch: subset of events_array with all mode switch triggers 
#     t_keystroke: subset of events_array with all keystrokes
#     ---
#     Returns: first keystrokes, a np array in the same format as events_array (3 columns, first column is time)
#     """
    
#     first_keystrokes = []
#     switch_times = t_modeswitch[:, 0]  # Extract mode switch times
#     switch_idx = 0
#     n_switches = len(switch_times)

    
#     keystroke_times = t_keystroke[:,0]

#     for keystroke in t_keystroke:
#         if switch_idx >= n_switches - 2:
#             break

#         ktime = keystroke[0]

#         #make sure the keystroke is between two mode switches
#         if ktime> switch_times[switch_idx] and ktime < switch_times[switch_idx+1]:
#             first_keystrokes.append(keystroke)
#             switch_idx+=1
        
#         #if there are no keystrokes between two mode switches, this forces it to jump to the next mode
#         elif ktime > switch_times[switch_idx+1] and ktime < switch_times[switch_idx+2]:
#             first_keystrokes.append(keystroke)
#             switch_idx+=2

#     return np.array(first_keystrokes)


In [362]:
def mapchange_keystrokes_4(t_modeswitch, t_keystroke):
    """ 
    Finds all the keystroke triggers that are the first keystrokes after a map change.

    t_modeswitch: subset of events_array with all mode switch triggers 
    t_keystroke: subset of events_array with all keystrokes
    ---
    Returns: first keystrokes, a np array in the same format as events_array (3 columns, first column is time)
    """
    
    first_keystrokes = []
    switch_times = t_modeswitch[:, 0]  # Extract mode switch times
    switch_idx = 0
    n_switches = len(switch_times)

    
    keystroke_times = t_keystroke[:,0]

    for keystroke in t_keystroke:
        if switch_idx >= n_switches - 2:  # Adjusted condition to avoid out-of-bounds
            break

        ktime = keystroke[0]

        # Make sure the keystroke is between two mode switches
        if ktime > switch_times[switch_idx] and ktime < switch_times[switch_idx + 1]:
            first_keystrokes.append(keystroke)
            switch_idx += 1

        # Skip consecutive mode switches until we find a keystroke in between
        else:
            try:
                while ktime > switch_times[switch_idx + 1]:
                    switch_idx += 1
            except IndexError:
                continue

            # If the keystroke is still valid after skipping switches, add it
            if ktime > switch_times[switch_idx] and ktime < switch_times[switch_idx + 1]:
                first_keystrokes.append(keystroke)
                switch_idx += 1

    return np.array(first_keystrokes)

firsts_test=mapchange_keystrokes_4(t_modeswitch, t_keystrokes)

In [363]:
def create_mapchange_df(first_keystrokes,  all_keystrokes):
    """ 
    Makes a df with the times and indices for first and other keystrokes
        for first keystrokes, further adds the number of keystrokes in the previous map
    returns: mapchange_df
    """

    
    mapchange_df = pd.DataFrame(columns = ['keystroke_idx', 'time', 'type', 'prev_keystrokes'])
    mapchange_df['time'] = all_keystrokes[:,0]

    mapchange_df['keystroke_idx'] = mapchange_df.index
    mapchange_df['type'] = mapchange_df['time'].apply(
            lambda x: 'first' if x in first_keystrokes[:, 0] 
            else 'other'
    )

    # Find the indices of rows where type is 'first'
    first_indices = mapchange_df.index[mapchange_df['type'] == 'first'].tolist()
    others_indices = mapchange_df.index[mapchange_df['type']=='other'].tolist()

    #For FIRSTS only: find the number of keystrokes before map change
    # Iterate over these indices and count 'other' keystrokes
    for i in range(len(first_indices)):
        current_index = first_indices[i]
        if i == 0:
            mapchange_df.at[current_index, 'prev_keystrokes'] = current_index
            #mapchange_df.at[current_index, 'prev_keystrokes'] = 0 #put 0 for previous keystrokes for the first map change (don't use)

        else:
            previous_index = first_indices[i - 1]
            # Count 'other' keystrokes between the two 'first' rows
            count_others = mapchange_df.loc[previous_index+1:current_index-1, 'type'].value_counts().get('other', 0)
            mapchange_df.at[current_index, 'prev_keystrokes'] = count_others
        
    
    # print(len(first_indices))
    #for OTHERS only: find the number of keystrokes since the last map change
            
    current_index_f = 1
    for current_index_o in others_indices:
        try:
            while current_index_o>first_indices[current_index_f]:
                current_index_f+=1

        except IndexError:
            current_index_f = current_index_f

        if current_index_f==1:
            mapchange_df.at[current_index_o, 'keystrokes_since'] = current_index_o
        else:
        
            others_counts = mapchange_df.loc[first_indices[current_index_f-1]:current_index_o, 'type'].value_counts().get('other', 0) -1
            mapchange_df.at[current_index_o, 'keystrokes_since'] = others_counts
    

    return mapchange_df

In [364]:

subjects_to_process =  ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#subjects_to_process = ['08']
periods = ['pre', 'post']
keystroke_trigs = 'audio'

overwrite = True #overwrite existing files
plot = False

find_mapchanges = True
find_modekeystrokes = True

#-----------------------------------------

#times for cropping ERPs
erp_begin = -0.5
erp_end = 0.5


#======================================================================================
#                        INITIALIZE DIRECTORIES
#======================================================================================
pp_dir = "/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/data_preprocessed_30Hz" #where the preprocessed files are
evokeds_folder = '/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/analysis_error/error_ERP_data_n05to05_30Hz'
epochs_folder = '/Users/cindyzhang/Documents/M2/Audiomotor_Piano/AM-EEG/analysis_error/error_epochs_data_n05to05_30Hz'
mapchange_dir = './mapchange_csv_corrected_since/'

if not os.path.exists(mapchange_dir):
    os.mkdir(mapchange_dir)


for folder in [evokeds_folder, epochs_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)
#======================================================================================
#                        CALCULATE EVOKEDS
#======================================================================================
for folder in sorted(os.listdir(pp_dir)):
    if folder not in subjects_to_process:
        continue

    print('\nPROCESSING SUBJECT ', folder)
    sub_pp_dir = os.path.join(pp_dir, folder)
    for period in periods:
        data_path = glob.glob(os.path.join(sub_pp_dir, f'eeg_error_{period}_??.mat'))[0]

        subject_ID = data_path.split('.')[0][-2:]

        print('Opening', data_path.split('/')[-1])
        

        #--------------------------------------------
        #               LOAD FILE AND EVENTS
        #--------------------------------------------
        data = loadmat(data_path)
        eeg = data['trial_data']
        refs = data['trial_mastoids']
        all_electrodes = data['all_electrodes']
        events_sv = data['events']

        if plot:
            mne.viz.plot_raw_psd(raw, fmin = 0, fmax = 64)  

        events_arr = make_raw_events(events_sv)

        if keystroke_trigs == 'MIDI':
            t_keystrokes = clean_triggers(events_arr[events_arr[:, 2]==6])
        else:
            t_keystrokes = clean_triggers(events_arr[events_arr[:, 2]==2])


        t_inv = clean_triggers(events_arr[events_arr[:, 2]==3])
        t_shinv = clean_triggers(events_arr[events_arr[:, 2]==4])
        t_norm = clean_triggers(events_arr[events_arr[:, 2]==5])
        t_modeswitch = np.concatenate([t_inv, t_shinv, t_norm])
        t_modeswitch = events_inorder(t_modeswitch)

        t_keystrokes_crop = np.array([x for x in t_keystrokes if x[0] > 64 and x[0]<76736])#crop out the keystrokes that are too close to the edge of recording

        first_keystrokes = mapchange_keystrokes_4(t_modeswitch = t_modeswitch, t_keystroke=t_keystrokes_crop)
        other_keystrokes = withinmap_keystrokes(t_keystrokes_crop, first_keystrokes)

        
        mapchange_df = create_mapchange_df(first_keystrokes, t_keystrokes_crop)
        mapchange_df.to_csv(os.path.join(mapchange_dir, f'mapchange_keystrokes_{period}_{subject_ID}.csv'))

        #print(first_keystrokes[-1])



PROCESSING SUBJECT  01
Opening eeg_error_pre_01.mat
Opening eeg_error_post_01.mat

PROCESSING SUBJECT  04
Opening eeg_error_pre_04.mat
Opening eeg_error_post_04.mat

PROCESSING SUBJECT  05
Opening eeg_error_pre_05.mat
Opening eeg_error_post_05.mat

PROCESSING SUBJECT  06
Opening eeg_error_pre_06.mat
Opening eeg_error_post_06.mat

PROCESSING SUBJECT  07
Opening eeg_error_pre_07.mat
Opening eeg_error_post_07.mat

PROCESSING SUBJECT  08
Opening eeg_error_pre_08.mat
Opening eeg_error_post_08.mat

PROCESSING SUBJECT  09
Opening eeg_error_pre_09.mat
Opening eeg_error_post_09.mat

PROCESSING SUBJECT  10
Opening eeg_error_pre_10.mat
Opening eeg_error_post_10.mat

PROCESSING SUBJECT  11
Opening eeg_error_pre_11.mat
Opening eeg_error_post_11.mat

PROCESSING SUBJECT  12
Opening eeg_error_pre_12.mat
Opening eeg_error_post_12.mat

PROCESSING SUBJECT  13
Opening eeg_error_pre_13.mat
Opening eeg_error_post_13.mat

PROCESSING SUBJECT  14
Opening eeg_error_pre_14.mat
Opening eeg_error_post_14.mat

PRO

In [365]:
# plt.figure(figsize = (25,10))
# plt.eventplot(t_inv[:, 0], lineoffsets = 3, color = 'green')
# plt.eventplot(t_shinv[:,0], lineoffsets = 2, color = 'orange')
# plt.eventplot(t_norm[:,0], lineoffsets = 1,color = 'red')
# plt.eventplot(t_modeswitch[:,0], lineoffsets = 0, color = 'grey')
# plt.eventplot(t_keystrokes[:,0], lineoffsets = -1, color = 'black')
# plt.eventplot(first_keystrokes[:,0], lineoffsets = -2, color = 'purple')
# plt.eventplot(other_keystrokes[:,0], lineoffsets = -3, color = 'steelblue')
# plt.xlim(0,10000)